In [63]:
import json

def read_data(filename):

  with open(filename, 'r') as json_file:
      json_list = list(json_file)

  dic = []
  for line in json_list:
    dic.append(json.loads(line))

  return dic

In [120]:
train_raw = read_data('train.jsonl')
dev_raw = read_data('dev.jsonl')
test_raw = read_data('test.jsonl')

In [121]:
train_raw[0]

{'text_id': 29620252,
 'Text': 'test fuck ejaculate mother cheez whiz ',
 'purity': 0,
 'harm': 0,
 'im': 1,
 'cv': 0,
 'ex': 0,
 'degradation': 0,
 'fairness': 0,
 'hd': 0,
 'mph': 0,
 'loyalty': 0,
 'care': 0,
 'betrayal': 0,
 'gen': 0,
 'cheating': 0,
 'subversion': 0,
 'rel': 0,
 'sxo': 0,
 'rae': 0,
 'nat': 0,
 'pol': 0,
 'authority': 0,
 'vo': 1,
 'idl': 0}

In [17]:
!wget https://strubell.github.io/teaching/11-830/assignments/files/identity_labels.txt

--2023-03-21 00:50:16--  https://strubell.github.io/teaching/11-830/assignments/files/identity_labels.txt
Resolving strubell.github.io (strubell.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to strubell.github.io (strubell.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1047 (1.0K) [text/plain]
Saving to: ‘identity_labels.txt’

identity_labels.txt 100%[===================>]   1.02K  --.-KB/s    in 0s      

2023-03-21 00:50:16 (24.9 MB/s) - ‘identity_labels.txt’ saved [1047/1047]



In [122]:
import pandas as pd

def read_BSW(filenames):

  identity_list = []
  with open(filenames[0], 'r') as f:
    lines= f.readlines()
    for line in lines:
      identity_list.append(line.rstrip())

  identity_list_2 = list(pd.read_csv(filenames[1], sep='\t')['muslim'])
  identity_list_2.append('muslim')

  identity_list_3 = list(pd.read_csv(filenames[2], sep='\t')['jew'])
  identity_list_2.append('jew')
  
  result = list(set(identity_list + identity_list_2 + identity_list_3))

  return result

In [123]:
identities = read_BSW(['identity_labels.txt', 'identity.csv', 'identity_ws_new.csv'])
len(identities)

140

In [ ]:
!pip install -U spacy

In [51]:
import spacy
nlp = spacy.load("en_core_web_sm")

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize

detokenizer = TreebankWordDetokenizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
from tqdm import tqdm

def to_pos(data):
  for k in tqdm(range(len(data))):
    sent = data[k]["Text"]
    text = word_tokenize(sent.lower())

    for i in range(len(text)):
      if text[i] in identities:
        text[i] = nlp(text[i])[0].pos_
        
    sent = detokenizer.detokenize(text)
    data[k]['Text'] = sent

  return data


In [131]:
def to_ner(data):
  for k in tqdm(range(len(data))):
    sent = data[k]["Text"]
    entities = nlp(sent.lower())
    text = [t.text if not t.ent_type_ else t.ent_type_ for t in entities]
    sent = detokenizer.detokenize(text)
    data[k]['Text'] = sent
  return data

In [95]:
train_data = train_raw.copy()
dev_data = dev_raw.copy()

train_data_pos = to_pos(train_data) # 963 969 971 973 980 986 997 998 1000 1006
dev_data_pos = to_pos(dev_data)

100%|██████████| 2581/2581 [00:01<00:00, 1906.82it/s]


In [145]:
train_data = train_raw.copy()
dev_data = dev_raw.copy()

train_data_ner = to_ner(train_data) # 5 6 7 8 9 11 12 23 24 25
dev_data_ner = to_ner(dev_data)

100%|██████████| 2581/2581 [00:35<00:00, 73.52it/s]


In [133]:
train_data_pos[963]['Text']

"a handful of intelligent & dedicated noun & noun can - & will - change the course of history . nineteen years ago, a few laid the foundation for what is now called 'populist nationalism .' others among us began to expose large scale corruption in government 100 such will rock the world to its core"

In [147]:
train_data_ner[6]['Text']

"didn't want the non - existent propn dirt .... just wanted to pay to find out how bad the propn screwed up .... that's all .... really...no really .... propn: punct"

In [ ]:
# sent = "How can these Liberal banks be allowed to dictate what LEGAL items consumers are allowed to purchase? What if the Liberals decide they don't like white shirts? And what about the debit card accounts that are prepaid with your own money? These companies should be REQUIRED BY LAW to complete ANY LEGAL transaction if they want a license to serve the public."
sent = 'She is a black woman'
sent = 'Mohan is a rock star of Hollywood'

# text = word_tokenize(sent.lower())
entities = nlp(sent)
text = [t.text if not t.ent_type_ else t.ent_type_ for t in entities]
sent = detokenizer.detokenize(text)
sent

In [142]:
def dump_file(filename, data):
  with open(filename, 'w') as f:
    for i in range(len(data)):
      json.dump(data[i], f)
      f.write('\n')

In [144]:
dump_file('train_pos.jsonl', train_data_pos)
dump_file('dev_pos.jsonl', dev_data_pos)

In [148]:
dump_file('train_ner.jsonl', train_data_ner)
dump_file('dev_ner.jsonl', dev_data_ner)